# Difference-in-Differences (DID) in Bioingegneria
In questo notebook analizziamo l'impatto di un intervento di bioingegneria, come l'utilizzo di un esoscheletro riabilitativo, sulla mobilità dei pazienti utilizzando il metodo Difference-in-Differences (DID).
Questo approccio è utile per stimare effetti causali confrontando le variazioni tra un gruppo trattato e un gruppo di controllo.

## Teoria del Difference-in-Differences
Il metodo DID stima l'effetto di un trattamento confrontando le differenze negli **outcome** medi tra gruppi e periodi.

### Formula Generale
L'effetto medio del trattamento sugli individui trattati (ATT) è dato da:
$$
ATT = \big(E[Y|D=1, Post=1] - E[Y|D=1, Post=0]\big) - \big(E[Y|D=0, Post=1] - E[Y|D=0, Post=0]\big)
$$

Dove:
- $E[Y|D=1, Post=1]$: Valore medio dell'outcome nel gruppo trattato dopo il trattamento.
- $E[Y|D=1, Post=0]$: Valore medio dell'outcome nel gruppo trattato prima del trattamento.
- $E[Y|D=0, Post=1]$: Valore medio dell'outcome nel gruppo di controllo dopo il trattamento.
- $E[Y|D=0, Post=0]$: Valore medio dell'outcome nel gruppo di controllo prima del trattamento.

### Ipotesi
Perché DID sia valido, devono essere rispettate alcune ipotesi:
- **Parallel Trend**: I due gruppi (controllo e trattato) avrebbero seguito la stessa traiettoria senza intervento.
- **Assenza di Spillover**: Il trattamento non influenza il gruppo di controllo.
- **Stabilità della composizione dei gruppi**: Non ci sono cambiamenti significativi nella composizione dei gruppi durante il periodo di studio.

## Simulazione dei Dati
Per illustrare l'approccio, genereremo un dataset simulato con due gruppi:
- **Gruppo di controllo**: Pazienti senza esoscheletro.
- **Gruppo trattato**: Pazienti che utilizzano l'esoscheletro.

Gli outcome rappresentano un punteggio di mobilità medio prima e dopo l'introduzione del trattamento.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Simulazione dei dati
np.random.seed(42)  # Fissiamo il seed per rendere i risultati riproducibili
n_patients = 200  # Numero totale di pazienti
print(f"Numero totale di pazienti simulati: {n_patients}")

# Creazione del dizionario dei dati
data = {
    'patient_id': np.arange(n_patients),  # ID univoco per ciascun paziente
    'treated': np.random.choice([0, 1], size=n_patients, p=[0.5, 0.5]),  # Gruppo di trattamento (1: trattato, 0: controllo)
    'pre_mobility': np.random.normal(50, 10, size=n_patients),  # Mobilità media prima del trattamento
    'post_mobility': None  # Placeholder per i dati di mobilità dopo il trattamento
}
print("Esempio dei dati iniziali (prime 5 righe):")
print(pd.DataFrame(data).head())

# Creazione del DataFrame
df = pd.DataFrame(data)

# Introduzione dell'effetto del trattamento
treatment_effect = 10  # Effetto del trattamento sulla mobilità
print(f"Effetto del trattamento impostato a: {treatment_effect}")

# Calcolo della mobilità post-intervento
# Aggiungiamo un effetto al gruppo trattato e introduciamo una variazione casuale
df['post_mobility'] = df['pre_mobility'] + np.where(df['treated'] == 1, treatment_effect, 0) + np.random.normal(0, 5, n_patients)
print("Esempio dei dati post trattamento (prime 5 righe):")
print(df[['patient_id', 'treated', 'pre_mobility', 'post_mobility']].head())

# Creazione della colonna 'post' per indicare il periodo
df['post'] = np.tile([0, 1], n_patients // 2)  # Alterniamo pre (0) e post (1) tra i pazienti
print("Aggiunta la colonna 'post' per indicare il periodo pre/post intervento.")
print(df[['patient_id', 'treated', 'post']].head())

# Mostriamo il dataset finale (prime righe)
print("\nDataset finale (prime 10 righe):")
print(df.head(10))

# Verifica delle statistiche descrittive per i gruppi
print("\nStatistiche descrittive per pre_mobility:")
print(df.groupby('treated')['pre_mobility'].describe())

print("\nStatistiche descrittive per post_mobility:")
print(df.groupby('treated')['post_mobility'].describe())


Numero totale di pazienti simulati: 200
Esempio dei dati iniziali (prime 5 righe):
   patient_id  treated  pre_mobility post_mobility
0           0        0     43.199753          None
1           1        1     52.322537          None
2           2        1     52.930725          None
3           3        1     42.856486          None
4           4        0     68.657745          None
Effetto del trattamento impostato a: 10
Esempio dei dati post trattamento (prime 5 righe):
   patient_id  treated  pre_mobility  post_mobility
0           0        0     43.199753      47.488051
1           1        1     52.322537      61.522844
2           2        1     52.930725      62.835644
3           3        1     42.856486      47.843839
4           4        0     68.657745      68.565179
Aggiunta la colonna 'post' per indicare il periodo pre/post intervento.
   patient_id  treated  post
0           0        0     0
1           1        1     1
2           2        1     0
3           3       

## Calcolo del Difference-in-Differences (DID)

### Obiettivo
Il calcolo del DID mira a stimare l'effetto medio del trattamento (ATT) confrontando le variazioni negli **outcome** medi tra i gruppi trattato e di controllo, nei periodi pre e post-intervento.

### Passaggi:
1. **Raggruppamento dei dati**:
   - Dividiamo il dataset in combinazioni di gruppo (trattato e controllo) e periodo (pre e post).
   - Calcoliamo la media degli **outcome** per ciascuna combinazione.

2. **Applicazione della formula del DID**:
   La formula generale per calcolare l'effetto medio del trattamento sugli individui trattati (ATT) è:
   $$
   ATT = \big(\bar{Y}_{T,Post} - \bar{Y}_{T,Pre}\big) - \big(\bar{Y}_{C,Post} - \bar{Y}_{C,Pre}\big)
   $$
   Dove:
   - $\bar{Y}_{T,Post}$: Media degli **outcome** nel gruppo trattato dopo l'intervento.
   - $\bar{Y}_{T,Pre}$: Media degli **outcome** nel gruppo trattato prima dell'intervento.
   - $\bar{Y}_{C,Post}$: Media degli **outcome** nel gruppo di controllo dopo l'intervento.
   - $\bar{Y}_{C,Pre}$: Media degli **outcome** nel gruppo di controllo prima dell'intervento.

3. **Interpretazione dei risultati**:
   - **ATT positivo**: Indica che l'intervento ha avuto un effetto positivo sul gruppo trattato.
   - **ATT negativo**: Indica che l'intervento ha avuto un effetto negativo sul gruppo trattato.
   - **ATT vicino a zero**: L'intervento non ha avuto effetti significativi rispetto al gruppo di controllo.

### Considerazioni:
- Il DID si basa sull'ipotesi di **Parallel Trend**, che implica che i due gruppi avrebbero seguito traiettorie simili in assenza di trattamento.
- Eventuali violazioni di questa ipotesi possono influenzare la validità dei risultati.


In [ ]:
# Calcolo delle medie per ogni combinazione di 'treated' e 'post'
print("\nCalcoliamo le medie per le combinazioni di gruppo (trattato/non trattato) e periodo (pre/post):")
did_data = (
    df.groupby(['treated', 'post'])
    .agg({'pre_mobility': 'mean', 'post_mobility': 'mean'})  # Calcoliamo le medie per pre e post-mobilità
)
print(did_data)

# Calcolo del controfattuale
# Media della mobilità pre-intervento per il gruppo trattato
treated_baseline = did_data.loc[(1, 0), 'pre_mobility']
print(f"\nBaseline del gruppo trattato (mobilità media pre-intervento): {treated_baseline:.2f}")

# Differenza nella mobilità nel gruppo di controllo (post - pre)
control_diff = did_data.loc[(0, 1), 'post_mobility'] - did_data.loc[(0, 0), 'pre_mobility']
print(f"Variazione della mobilità nel gruppo di controllo: {control_diff:.2f}")

# Controfattuale per il gruppo trattato (baseline + variazione del controllo)
counterfactual = treated_baseline + control_diff
print(f"Stima del controfattuale per il gruppo trattato: {counterfactual:.2f}")

# Calcolo dell'ATT (effetto medio del trattamento)
# Differenza tra il risultato osservato nel gruppo trattato post-intervento e il controfattuale
att = did_data.loc[(1, 1), 'post_mobility'] - counterfactual
print(f"\nEffetto Medio del Trattamento (ATT): {att:.2f}")



Calcoliamo le medie per le combinazioni di gruppo (trattato/non trattato) e periodo (pre/post):
              pre_mobility  post_mobility
treated post                             
0       0        50.095920      50.885519
        1        50.708852      49.329724
1       0        51.564584      60.423755
        1        50.385962      60.907971

Baseline del gruppo trattato (mobilità media pre-intervento): 51.56
Variazione della mobilità nel gruppo di controllo: -0.77
Stima del controfattuale per il gruppo trattato: 50.80

Effetto Medio del Trattamento (ATT): 10.11


### Interpretazione del Risultato

L'interpretazione del valore stimato per l'**ATT** (Effetto Medio del Trattamento sugli individui trattati) fornisce indicazioni importanti sull'efficacia dell'intervento.

- **Significato dell'ATT**:
  - Il valore stimato rappresenta la differenza media negli **outcome** attribuibile al trattamento, confrontando il gruppo trattato con il gruppo di controllo.

- **Risultati Possibili**:
  1. **ATT positivo**:
     - Indica che il trattamento ha migliorato l'**outcome** (es. mobilità) del gruppo trattato rispetto al gruppo di controllo.
     - L'intervento può essere considerato efficace.
  2. **ATT negativo**:
     - Indica che il trattamento ha ridotto l'**outcome** del gruppo trattato rispetto al gruppo di controllo.
     - Questo potrebbe suggerire effetti negativi o indesiderati del trattamento.
  3. **ATT vicino a zero**:
     - Suggerisce che il trattamento non ha avuto un impatto significativo rispetto al gruppo di controllo.
     - Potrebbero essere necessari ulteriori studi per comprendere le ragioni di questo risultato.

- **Considerazioni Aggiuntive**:
  - I risultati devono essere interpretati alla luce delle ipotesi del metodo Difference-in-Differences, in particolare l'ipotesi di **Parallel Trend**.
  - Eventuali confondenti o violazioni delle ipotesi potrebbero influenzare la stima dell'ATT e la sua interpretazione.


## Visualizzazione dei Risultati

La visualizzazione è fondamentale per comprendere meglio i risultati ottenuti dall'analisi Difference-in-Differences (DID). In questa sezione, creiamo un grafico che evidenzia:

1. **Risultati Osservati**:
   - La traiettoria degli **outcome** medi nel tempo per il gruppo trattato e il gruppo di controllo, sia prima che dopo l'intervento.
   - Questi dati rappresentano le osservazioni reali ottenute dal dataset.

2. **Stima del Controfattuale**:
   - La traiettoria stimata per il gruppo trattato in assenza dell'intervento, calcolata utilizzando la formula del DID.
   - Il controfattuale è una previsione teorica basata sull'ipotesi di Parallel Trend.

### Elementi del Grafico
- **Linea per il gruppo trattato (osservato)**:
  Mostra l'andamento reale degli **outcome** per i pazienti trattati nei periodi pre e post-intervento.
- **Linea per il gruppo di controllo (osservato)**:
  Rappresenta la traiettoria reale degli **outcome** per i pazienti di controllo nei periodi pre e post-intervento.
- **Linea per il controfattuale (stimato)**:
  Mostra la traiettoria attesa per il gruppo trattato in assenza di intervento. Questa linea è fondamentale per visualizzare l'effetto stimato del trattamento.

### Interpretazione
- La differenza tra il **risultato osservato** e il **controfattuale stimato** nel periodo post-intervento rappresenta il valore dell'**ATT**.
- Un'analisi visiva consente di verificare se i dati rispettano l'ipotesi di Parallel Trend (ovvero traiettorie parallele prima dell'intervento).

Questa rappresentazione grafica facilita la comprensione sia dei risultati numerici che delle ipotesi alla base del modello.


In [ ]:
import plotly.graph_objects as go

# Visualizzazione dei risultati con Difference-in-Differences
print("\nVisualizzazione dei risultati:")

# Preparazione dei dati
treated_observed = [did_data.loc[(1, 0), 'pre_mobility'], did_data.loc[(1, 1), 'post_mobility']]
control_observed = [did_data.loc[(0, 0), 'pre_mobility'], did_data.loc[(0, 1), 'post_mobility']]
treated_counterfactual = [did_data.loc[(1, 0), 'pre_mobility'], counterfactual]

print(f"Dati osservati - Gruppo trattato: Pre-Intervento = {did_data.loc[(1, 0), 'pre_mobility']:.2f}, Post-Intervento = {did_data.loc[(1, 1), 'post_mobility']:.2f}")
print(f"Dati osservati - Gruppo di controllo: Pre-Intervento = {did_data.loc[(0, 0), 'pre_mobility']:.2f}, Post-Intervento = {did_data.loc[(0, 1), 'post_mobility']:.2f}")
print(f"Dati stimati - Controfattuale gruppo trattato: Pre-Intervento = {did_data.loc[(1, 0), 'pre_mobility']:.2f}, Post-Intervento = {counterfactual:.2f}")

# Creazione del grafico Plotly
fig = go.Figure()

# Linea per il gruppo trattato (osservato)
fig.add_trace(go.Scatter(
    x=['Pre-Intervento', 'Post-Intervento'],
    y=treated_observed,
    mode='lines+markers',
    name='Trattato (osservato)',
    line=dict(dash='solid', color='blue'),
    marker=dict(size=8)
))

# Linea per il gruppo di controllo (osservato)
fig.add_trace(go.Scatter(
    x=['Pre-Intervento', 'Post-Intervento'],
    y=control_observed,
    mode='lines+markers',
    name='Controllo (osservato)',
    line=dict(dash='solid', color='green'),
    marker=dict(size=8)
))

# Linea per il controfattuale stimato del gruppo trattato
fig.add_trace(go.Scatter(
    x=['Pre-Intervento', 'Post-Intervento'],
    y=treated_counterfactual,
    mode='lines+markers',
    name='Trattato (controfattuale)',
    line=dict(dash='dash', color='red'),
    marker=dict(size=8, symbol='x')
))

# Configurazione del layout
fig.update_layout(
    title="Difference-in-Differences (DID)",
    xaxis=dict(title="Periodo"),
    yaxis=dict(title="Mobilità Media"),
    legend=dict(title="Gruppo"),
    template="plotly_white",
    margin=dict(l=50, r=50, t=50, b=50),
    height=500
)

# Mostra il grafico
fig.show()



Visualizzazione dei risultati:
Dati osservati - Gruppo trattato: Pre-Intervento = 51.56, Post-Intervento = 60.91
Dati osservati - Gruppo di controllo: Pre-Intervento = 50.10, Post-Intervento = 49.33
Dati stimati - Controfattuale gruppo trattato: Pre-Intervento = 51.56, Post-Intervento = 50.80
